# **Chapter 3: Assigning Roles (Role Prompting)**

---
**Lesson:**

When working with Llama3, it's important to provide detailed context to guide its responses. One effective technique is role prompting, where you instruct Llama3 to adopt a specific role with all necessary context. The more detailed the role, the better the results.

By priming Llama3 with a specific role, you can enhance its performance across various tasks, from writing and coding to summarizing. Think of it as giving Llama3 a clear perspective, similar to how humans benefit from being told to "think like a ______."

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

modelId = 'meta.llama3-8b-instruct-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

def invoke_model_and_get_response(prompt_data): 
    body = json.dumps({ 
        'prompt': prompt_data,
        'max_gen_len': 1024,
        'top_p': 0.9,
        'temperature': 0.2
    })

    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read().decode('utf-8'))
        outputText = response_body['generation'].strip()
        return outputText

    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'AccessDeniedException':
            return (f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubleshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        else:
            raise

# **Examples:**

In the example below (3.1), we see that without priming, Llama3 can provide a decent if somewhat disjointed answer when asked to explain the impact of greenhouse gases. 

However, when we prime Llama3 to be a highly experienced climate scientist, Llama3's response becomes more organized and detailed (3.2). 

> *Note: A bonus technique you can use is to provide Llama3 context on its intended audience. Below, we could have tweaked the prompt to also tell Llama3 whom it should be speaking to. "Explain the impact of greenhouse gasses" produces very different results in terms of tone from "Explain the impact of greenhouse gasses to a third grade audience."*

**Example 3.1 - Greenhouse**

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
Please explain the impact of greenhouse gases on the planet.
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 3.2 - Organized greenhouse**

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a highly experienced climate scientist.
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
Please explain the impact of greenhouse gases on the planet.
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 3.3 - Logical?**

Here's another example. Let's ask Llama to solve this logical problem

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don't know if Anne is married. Is a married person looking at an unmarried person?
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 3.4 - Logical!**

The answer above is surface level reasoning based on the information given. Let us give Llama3 a role as a "Logic Bot" and see how it's response changes

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a logic bot designed to answer complex logic problems by thinking step-by-step.
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don't know if Anne is married. Is a married person looking at an unmarried person?
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 3.1 - Math Correction**

In some instances, Llama3 may struggle with mathematics, even simple mathematics. Below, Claude incorrectly assesses the math problem as correctly solved, even though there's an obvious arithmetic mistake in the second step.

Adapt the prompt below to make Claude grade the solution as "incorrectly" solved, rather than "correctly" solved. 

> *"Note: XML tags (<> to open, </> to close) are a common and highly effective way to organize information within a prompt. Sometimes it also helps to ask Claude to think about its answer first before responding. 
We will cover these two topics in greater depth in Chapters 4 and 6 respectively."*

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Assume you are a world class mathematician. Is this equation solved correctly below?
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
2x - 3 = 9
2x = 6 
x = 3
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

# Chapter 3 - END.